# Monte Carlo Pi

In this sample solution, the points are divided amonst all of the processes as evenly as possible. `count_points_inside_circle` is called in each process and returns the number of its points that are inside the circle by adding them to a `Queue`. These values are retrieved from the `Queue` (note that we don't need to wait with `join` as `Queue.get` waits for each item to be added to the `Queue`). The total of these values is calcualted, and then used to estimate a value of $\pi$.

In [1]:
import random
import time
import multiprocessing

# Define the function to be called by each process
def count_points_inside_circle(n_points, queue):
    # Calculate the number of points inside the circle and adds it to the queue

    # Initiate the variable to store the number of points inside the circle
    n_inside_local = 0
    for i in range(n_points):
        # Generate a random point
        x = random.random()
        y = random.random()

        if x ** 2 + y ** 2 <= 1:
            # If the point is inside the circle, increment the counter
            n_inside_local += 1

    # Add the number of points inside the circle to the queue
    queue.put(n_inside_local)

# Note the starting time
start_time = time.time()

# Number of points to generate
n_points = int(1e7)

# Number of processes to use
n_processes = 4

# Create the queue
queue = multiprocessing.Queue()

for i in range(n_processes):
    # Start the processes
    p = multiprocessing.Process(target=count_points_inside_circle, args=(n_points // n_processes, queue))
    p.start()

# Initiate the variable to store the total number of points inside the circle
n_points_inside_circle = 0

for i in range(n_processes):
    # Add the number of points inside the circle
    # There is no need to wait for the processes to finish as get blocks until the result is available
    n_points_inside_circle += queue.get()

# Calculate the value of pi
pi_approximation = 4 * n_points_inside_circle / n_points

print(f'The value of pi is approximately {pi_approximation}')

print(f'Time taken: {time.time() - start_time}')

The value of pi is approximately 3.142074
Time taken: 2.6541197299957275


# Approximating Sine

The core to this solution is the creation of `sine_term` a function which calcualtes the `k`th term of the approximation for a given value of `x`. As this has two parameters, it can be called using a `starmap`. This requires constructing the list of lists `inputs` in the function `sine_approx` which contains the arguments to be passed to the function each time. The `starmap` is implemented using a `Pool` and the returned values are summed to give the final answer.

In [2]:
from multiprocessing import Pool
from math import factorial, pi, sin

def sine_term(x, k):
    # This function calculates the nth term of the Taylor series expansion of sine(x)
    # x is the value at which to evaluate the sine function
    # k is the index of the term to calculate

    # Calculate and return the term
    return ((-1) ** k) * (x ** (2 * k + 1)) / factorial(2 * k + 1)

def sine_approx(x, n_terms):
    # This function calculates an approximation of the sine function at x using n_terms terms of the Taylor series expansion

    # Create a list of inputs for the sine_term function
    inputs = [(x, k) for k in range(n_terms)]
    print(inputs)

    # Create a pool of two threads to calculate the terms in parallel
    with Pool(2) as p:
        # Calculate the terms in parallel and store the results in a list
        terms = p.starmap(sine_term, inputs)

    # Return the sum of the terms
    return sum(terms)

# Test the sine_approx function
x = 0.5
n_terms = 3
print(sine_approx(x, n_terms), sin(x))

x = pi
n_terms = 5
print(sine_approx(x, n_terms), sin(x))

[(0.5, 0), (0.5, 1), (0.5, 2)]
0.47942708333333334 0.479425538604203
[(3.141592653589793, 0), (3.141592653589793, 1), (3.141592653589793, 2), (3.141592653589793, 3), (3.141592653589793, 4)]
0.006925270707505135 1.2246467991473532e-16
